In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('ufc_fights.csv')
# print(df.columns)
print(', '.join(df.columns))
# print(df.head())
# print(df.info())
# print(df.describe())

RedFighter, BlueFighter, RedOdds, BlueOdds, RedExpectedValue, BlueExpectedValue, Date, Location, Country, Winner, TitleBout, WeightClass, Gender, NumberOfRounds, BlueCurrentLoseStreak, BlueCurrentWinStreak, BlueDraws, BlueAvgSigStrLanded, BlueAvgSigStrPct, BlueAvgSubAtt, BlueAvgTDLanded, BlueAvgTDPct, BlueLongestWinStreak, BlueLosses, BlueTotalRoundsFought, BlueTotalTitleBouts, BlueWinsByDecisionMajority, BlueWinsByDecisionSplit, BlueWinsByDecisionUnanimous, BlueWinsByKO, BlueWinsBySubmission, BlueWinsByTKODoctorStoppage, BlueWins, BlueStance, BlueHeightCms, BlueReachCms, BlueWeightLbs, RedCurrentLoseStreak, RedCurrentWinStreak, RedDraws, RedAvgSigStrLanded, RedAvgSigStrPct, RedAvgSubAtt, RedAvgTDLanded, RedAvgTDPct, RedLongestWinStreak, RedLosses, RedTotalRoundsFought, RedTotalTitleBouts, RedWinsByDecisionMajority, RedWinsByDecisionSplit, RedWinsByDecisionUnanimous, RedWinsByKO, RedWinsBySubmission, RedWinsByTKODoctorStoppage, RedWins, RedStance, RedHeightCms, RedReachCms, RedWeightLb

In [ ]:
#analyze and handle missing data

def missing_value_analysis(df):
    missing_values = df.isnull().sum()
    missing_values_percentage = (missing_values / len(df)) * 100
    missing_values_table = pd.concat([missing_values, missing_values_percentage], axis=1)
    missing_values_table = missing_values_table.rename(columns={0: 'Missing Values', 1: 'Percentage'})

    return missing_values_table[missing_values_table['Missing Values'] > 0].sort_values(by='Percentage', ascending=False)


missing_data = missing_value_analysis(df)
print(missing_data)